# Audio Model

In [2]:
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from common.metrics import f1_m, recall_m, precision_m

In [3]:
audio_data_fp = "../mmsd_raw_data/converted_utterances"
dataset_csv_path = "../normalized_mustard_dataset.csv"

In [4]:
def create_mfcc(filepath:str, n_fft:int, hop_length: int, n_mfcc:int = 13):
    """
    Creates MFCC for file at filepath

    :param filepath: Location of file to be used
    :param n_fft: Number of Fast Fourier Transforms
    :param hop_length: Number of Hops within samples
    :param n_mfcc: Number of MFCC's to be outputted
    :return: Array containing mean of all MFCC's
    """
    signal, sample_rate = librosa.load(filepath)
    mfccs = librosa.feature.mfcc(y=signal, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
    mean_mfccs = np.mean(mfccs.T,axis=0)
    return mean_mfccs

In [5]:
def create_mfcc_features(data):
    """
    Creates a list of Mel-Frequency Co-Efficients
    :param data: Pandas Dataframe of Input Data
    """
    hop_length = 512
    n_fft = 2048

    _mfcc_df = pd.DataFrame(columns=["features", "sarcasm_state"])

    tqdm_data = tqdm(zip(data["file_name"], data["sarcasm"]))

    for file_name,sarcasm_state in tqdm_data:
        tqdm_data.set_description(f"Creating MFCC for {file_name}")
        fp = f"{audio_data_fp}/{str(file_name)}"
        mean_mfccs = create_mfcc(fp, n_fft, hop_length)
        _mfcc_df = _mfcc_df.append({
            "features": mean_mfccs,
            "sarcasm_state": sarcasm_state
        }, ignore_index=True)

    return _mfcc_df

In [6]:
dataset_df = pd.read_csv(dataset_csv_path)
mfcc_df = create_mfcc_features(dataset_df)

Creating MFCC for 2_524.wav: : 690it [02:42,  4.25it/s]  


In [7]:
mfcc_df.head()

,features,sarcasm_state
0,"[-146.32887, 99.83219, -50.631638, 9.274939, -...",True
1,"[-147.90367, 104.595116, -39.535, 4.108271, -2...",True
2,"[-21.120022, 86.11223, -35.381474, 22.640253, ...",False
3,"[-23.78374, 70.78856, -34.25742, 23.38274, -16...",False
4,"[1.6350226, 82.73289, -46.96253, 14.140306, -2...",True


In [8]:
X = np.array(mfcc_df['features'].tolist())
y = np.array(mfcc_df['sarcasm_state'].tolist())

In [9]:
X.shape

(690, 13)

In [10]:
# Label Encoder for getting sarcasm state
label_encoder = LabelEncoder()
y = to_categorical(label_encoder.fit_transform(y))

In [11]:
y.shape

(690, 2)

In [12]:
X_audio_train, X_audio_test, y_audio_train, y_audio_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
# Number of Classes
label_count = y.shape[1]

In [14]:
model = Sequential()
model.add(Dense(100, input_shape=(13,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(label_count))
model.add(Activation('softmax'))

model.summary()

2022-05-03 00:13:44.804497: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-03 00:13:44.806501: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1400      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [15]:
model.compile(loss="categorical_crossentropy", metrics=['accuracy',f1_m, precision_m, recall_m], optimizer='adam')

In [16]:
num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath="../models/audio_model.h5", verbose=True, save_best_only=True)

model.fit(X_audio_train, y_audio_train, batch_size=num_batch_size, epochs=num_epochs, callbacks=[checkpointer])

Train on 552 samples
Epoch 1/100
552/552 [==============================] - 1s 3ms/sample - loss: 29.9158 - accuracy: 0.5000 - f1_m: 0.5052 - precision_m: 0.5052 - recall_m: 0.5052
Epoch 2/100
552/552 [==============================] - 0s 187us/sample - loss: 17.6933 - accuracy: 0.5272 - f1_m: 0.5260 - precision_m: 0.5260 - recall_m: 0.5260
Epoch 3/100
552/552 [==============================] - 0s 177us/sample - loss: 12.1201 - accuracy: 0.5254 - f1_m: 0.5347 - precision_m: 0.5347 - recall_m: 0.5347
Epoch 4/100
552/552 [==============================] - 0s 177us/sample - loss: 8.8699 - accuracy: 0.4909 - f1_m: 0.5017 - precision_m: 0.5017 - recall_m: 0.5017
Epoch 5/100
552/552 [==============================] - 0s 176us/sample - loss: 6.0611 - accuracy: 0.5562 - f1_m: 0.5590 - precision_m: 0.5590 - recall_m: 0.5590
Epoch 6/100
552/552 [==============================] - 0s 185us/sample - loss: 5.1719 - accuracy: 0.5417 - f1_m: 0.5347 - precision_m: 0.5347 - recall_m: 0.5347
Epoch 7/100


## Audio Model Testing

In [17]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_audio_test, y_audio_test, verbose=0)
print(f"loss: {loss}\naccuracy:{accuracy}\nf1_score:{f1_score}\nprecision:{precision}\nrecall:{recall}")

loss: 0.6340242900710175
accuracy:0.6159420013427734
f1_score:0.6274999380111694
precision:0.6274999976158142
recall:0.6274999976158142


In [18]:
test_file_path = "../mmsd_raw_data/converted_utterances/2_626.wav"
audio, sample_rate = librosa.load(test_file_path)
mfcc_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
mfcc_scaled_features = np.mean(mfcc_features.T, axis=0)

mfcc_scaled_features = mfcc_scaled_features.reshape(1, -1)

In [19]:
predicted_label = model.predict_classes(mfcc_scaled_features)
predicted_class = label_encoder.inverse_transform(predicted_label)
print(f"Sarcastic: {predicted_class[0]}")

Sarcastic: False
